# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

Make sure you have the latest version of GraphLab Create.

In [2]:
from __future__ import division
import graphlab
import math
import string
print "packages imported___"

packages imported___


# Data preparation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [3]:
products = graphlab.SFrame('../data/amazon_baby.gl/')
print products.shape

This non-commercial license of GraphLab Create for academic use is assigned to bmatthewtaylor@gmail.com and will expire on October 27, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\username\AppData\Local\Temp\graphlab_server_1484111766.log.0


(183531, 3)


Now, let us see a preview of what the dataset looks like.

In [4]:
print products.column_names()
print products.column_types()
products

['name', 'review', 'rating']
[<type 'str'>, <type 'str'>, <type 'float'>]


name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [5]:
products[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [6]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 
print "function remove_punctuation created"
#https://docs.python.org/2/library/string.html
#NB: string.translate is deprecated in python 2.x will be removed in python 3.
#string.translate(s, table[, deletechars])
#Delete all characters from s that are in deletechars (if present), and then 
#translate the characters using table, which must be a 256-character string 
#giving the translation for each character value, indexed by its ordinal. 
#If table is None, then only the character deletion step is performed.
#
#string.punctuation 
#String of ASCII characters which are considered punctuation characters in the C locale.
#
review_without_punctuation = products['review'].apply(remove_punctuation)
products['word_count'] = graphlab.text_analytics.count_words(review_without_punctuation)
#NB: this is slow to execute.

function remove_punctuation created


Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [7]:
print products[269]['word_count']
print products[269]['review']

{'house': 1L, 'our': 1L, 'favorite': 1L, 'a': 1L, 'in': 1L}
A favorite in our house!


## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [8]:
print products.shape[0]#originally was 183531 rows.
sentimentsNeutral = products[products['rating'] == 3]
products = products[products['rating'] != 3]
print len(products)
#nb: we just culled products['rating'] == 3 from products
print "--------"

183531
166752
--------


Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [9]:
print products.column_names()

['name', 'review', 'rating', 'word_count']


In [10]:
#add column 'sentiment' & populate with values.
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3L, 'love': 1L,'it': 3L, 'highly': 1L, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2L, 'quilt': 1L,'it': 1L, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'and': 3L, 'ingenious':1L, 'love': 2L, 'is': ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2L, 'all': 2L,'help': 1L, 'cried': 1L, ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2L, 'cute': 1L,'help': 2L, 'habit': 1L, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1L, 'be': 1L,'is': 1L, 'bound': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2L, 'all': 1L,'right': 1L, 'able': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1L, 'fantastic':1L, 'help': 1L, 'give': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'all': 1L, 'standarad':1L, 'another': 1L, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 2L, 'nannys': 1L,'just': 1L, 'sleep': 2L, ...",1


In [11]:
sentimentPos = products.filter_by( 1, 'sentiment').shape[0]
sentimentNeg = products.filter_by(-1, 'sentiment').shape[0]
print " neutral sentiments = ", sentimentsNeutral.shape[0]
print "positive sentiments = ", sentimentPos
print "negative sentiments = ", sentimentNeg
print "total nonzero sentiments = ", sentimentPos + sentimentNeg
#NB: many more positive than negative.

 neutral sentiments =  16779
positive sentiments =  140259
negative sentiments =  26493
total nonzero sentiments =  166752


Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [14]:
train_data, test_data = products.random_split(.8, seed=1)
print "split completed"

split completed


In [18]:
print type(train_data)
print type(test_data)
print "train_data.shape = ", train_data.shape
print "test_data.shape = ", test_data.shape

<class 'graphlab.data_structures.sframe.SFrame'>
<class 'graphlab.data_structures.sframe.SFrame'>
train_data.shape =  (133416, 5)
test_data.shape =  (33336, 5)


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [19]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'sentiment',
                                                      features=['word_count'],
                                                      validation_set=None)
#https://turi.com/products/create/docs/generated/graphlab.logistic_classifier.create.html
#solver : string, optional
#Name of the solver to be used to solve the regression. See the references for more detail on each solver. 
#Available solvers are: auto (default): automatically chooses the best solver for the data and model parameters.
#since we did not nominate solver type, best type chosen, sentiment_model (below) shows us what is used.

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 6.957259     | 0.960964          |

| 6         | 13       | 3.000000  | 7.815028     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [20]:
sentiment_model
#https://turi.com/products/create/docs/generated/graphlab.logistic_classifier.LogisticClassifier.html


Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 6
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 8.4706

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
word_count[mobileupdate]       : 41.9847
word_count[placeid]            : 41.7354
word_count[labelbox]           : 41.151
word_count[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid] : 40.0454
word_count[knobskeeping]       : 36.2091

Lowest Negative Coeffi

**Aside**. You may get a warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [21]:
weights = sentiment_model.coefficients
print "weights.column_names() = ", weights.column_names()

weights.column_names() =  ['name', 'index', 'class', 'value', 'stderr']


There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [22]:
num_positive_weights = weights[weights['value']>0].shape[0]
num_negative_weights = weights[weights['value']<0].shape[0]
num_zero_weights = weights[weights['value']==0].shape[0]


print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights
print "Number of weights == 0: %s "     % num_zero_weights

Number of positive weights: 68419 
Number of negative weights: 53294 
Number of weights == 0: 0 


**Quiz Question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [23]:
sample_test_data = test_data[10:13]
print "sample_test_data['rating']= ", sample_test_data['rating']
sample_test_data

sample_test_data['rating']=  [5.0, 2.0, 1.0]


name,review,rating,word_count,sentiment
Our Baby Girl Memory Book,Absolutely love it andall of the Scripture in ...,5.0,"{'and': 2L, 'all': 1L,'love': 1L, ...",1
Wall Decor RemovableDecal Sticker - Colorful ...,Would not purchase againor recommend. The decals ...,2.0,"{'and': 1L, 'wall': 1L,'them': 1L, 'decals': ...",-1
New Style Trailing CherryBlossom Tree Decal ...,Was so excited to getthis product for my baby ...,1.0,"{'all': 1L, 'money': 1L,'into': 1L, 'it': 3L, ...",-1


Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [24]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [25]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [26]:
#recall : sample_test_data = test_data[10:13]
scores = sentiment_model.predict(sample_test_data, output_type='margin')
print type(scores)
print scores

<class 'graphlab.data_structures.sarray.SArray'>
[6.7346197270597195, -5.734130996760575, -14.668460404469018]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [27]:
def classPredict(scores):
    #scores [graphlab.data_structures.sarray.SArray] : contains scores of dtype float
    #returns list of predicted classes +1 or -1
    predictedClass = []
    for score in scores:
        if score >=0:
            #NB this includes >0 _&_ ==0 case
            predictedClass.append(1)
        else:
            predictedClass.append(-1)
    return predictedClass
print "function classPredict loaded."
result1 =  classPredict(scores)
print type(result1)
print result1
#todo: should be able to rewrite this as a more compact lambda function

function classPredict loaded.
<type 'list'>
[1, -1, -1]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [32]:
print "Class predictions according to GraphLab Create:" 
result2 = sentiment_model.predict(sample_test_data)
print result2
print type(list(result2))
#NB: checking lists are equal is tricky. using lazy visual comparison for now.
equalLists = True
for i in range(len(result2)):
    if result1[i] != result2[i]:
        equalLists = False
print "classifications are same via both methods : ", equalLists

Class predictions according to GraphLab Create:
[1L, -1L, -1L]
<type 'list'>
classifications are same via both methods :  True


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [33]:
def probPredictions(scores):
    # scores [graphlab.data_structures.sarray.SArray] scores are dtype float.
    #returns list of calculated probabilities.
    probabilities = []
    for score in scores:
        probabilities.append(1/(1+math.exp(-score)))
    return probabilities
probPredictions(scores)

[0.9988123848377203, 0.0032232681817996617, 4.261557996653293e-07]

**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [118]:
print "Class predictions according to GraphLab Create:" 
probabilities = sentiment_model.predict(sample_test_data, output_type='probability')
probabilities = list(probabilities)
print type(probabilities)
minIndex = probabilities.index(min(probabilities))
print "index of min position = ", minIndex
print "probabilities = ", probabilities
print sample_test_data[minIndex]['review']

Class predictions according to GraphLab Create:
<type 'list'>
index of min position =  2
probabilities =  [0.9988123848377203, 0.0032232681817996604, 4.261557996653291e-07]
Was so excited to get this product for my baby girls bedroom!  When I got it the back is NOT STICKY at all!  Every time I walked into the bedroom I was picking up pieces off of the floor!  Very very frustrating!  Ended up having to super glue it to the wall...very disappointing.  I wouldn't waste the time or money on it.


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [35]:
#recall how key variables were created.
#sentiment_model = graphlab.logistic_classifier.create(train_data, target = 'sentiment', features=['word_count'], validation_set=None)
#recall train_data, test_data = products.random_split(.8, seed=1)
#predict probabilities and store in new column in test_data
test_data['prob'] = sentiment_model.predict(test_data, output_type='probability')
print test_data.shape, test_data.column_names()

(33336, 6) ['name', 'review', 'rating', 'word_count', 'sentiment', 'prob']


In [36]:
#https://turi.com/products/create/docs/generated/graphlab.SFrame.topk.html
test_data.topk('prob', k=20).print_rows(num_rows=20)


+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Britax Decathlon Convertib... | I researched a few differe... |  4.0   |
| Ameda Purely Yours Breast ... | As with many new moms, I c... |  4.0   |
| Traveling Toddler Car Seat... | I am sure this product wor... |  2.0   |
| Shermag Glider Rocker Comb... | After searching in stores ... |  4.0   |
| Cloud b Sound Machine Soot... | First off, I love plush sh... |  5.0   |
| JP Lizzy Chocolate Ice Cla... | I got this bag as a presen... |  4.0   |
| Fisher-Price Rainforest Me... | My daughter wasn't able to... |  5.0   |
| Lilly Gold Sit 'n' Stroll ... | I just completed a two-mon... |  5.0   |
|  Fisher-Price Deluxe Jumperoo | I had already decided that... |  5.0   |
| North States Supergate Pre... | I got a couple of these to... |  4.0   |
|   Munchkin Mozart Magic

In [119]:
#print in user friendlier format for quiz question purposes.
result = test_data.topk('prob', k=20, reverse = False)['name']
print result.shape
for i in range(result.shape[0]):
    print result[i]

(20,)
Britax Decathlon Convertible Car Seat, Tiffany
Ameda Purely Yours Breast Pump - Carry All
Traveling Toddler Car Seat Travel Accessory
Shermag Glider Rocker Combo, Pecan with Oatmeal
Cloud b Sound Machine Soother, Sleep Sheep
JP Lizzy Chocolate Ice Classic Tote Set
Fisher-Price Rainforest Melodies and Lights Deluxe Gym
Lilly Gold Sit 'n' Stroll 5 in 1 Car Seat and Stroller Combination, Tuxedo Black (sunshade is not included in the offering)
Fisher-Price Deluxe Jumperoo
North States Supergate Pressure Mount Clear Choice Wood Gate
Munchkin Mozart Magic Cube
Britax Marathon Convertible Car Seat, Granite
Wizard Convertible Car Seat with LATCH in Midnight Print
Capri Stroller - Red Tech
Peg Perego Primo Viaggio Car Seat / Infant Carrier with LATCH Base - Black Sable
HALO SleepSack Micro-Fleece Wearable Blanket, Soft Pink, Small
Leachco Snoogle Total Body Pillow
Summer Infant Complete Nursery Care Kit
Safety 1st Tot-Lok Four Lock Assembly
BABYBJORN Potty Chair - Red


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [37]:
test_data.topk('prob', k=20, reverse = True).print_rows(num_rows=20)

+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
| Jolly Jumper Arctic Sneak ... | I am a "research-aholic" i... |  5.0   |
| Levana Safe N'See Digital ... | This is the first review I... |  1.0   |
| Snuza Portable Baby Moveme... | I would have given the pro... |  1.0   |
| Fisher-Price Ocean Wonders... | We have not had ANY luck w... |  2.0   |
| VTech Communications Safe ... | This is my second video mo... |  1.0   |
| Safety 1st High-Def Digita... | We bought this baby monito... |  1.0   |
| Chicco Cortina KeyFit 30 T... | My wife and I have used th... |  1.0   |
| Prince Lionheart Warmies W... | *****IMPORTANT UPDATE*****... |  1.0   |
| Valco Baby Tri-mode Twin S... | I give one star to the dim... |  1.0   |
| Adiri BPA Free Natural Nur... | I will try to write an obj... |  2.0   |
| Munchkin Nursery Projec

In [46]:
#print in user friendlier format for quiz question purposes.
result = test_data.topk('prob', k=20, reverse = True)['name']
print result.shape
for i in range(result.shape[0]):
    print result[i]

(20,)
Jolly Jumper Arctic Sneak A Peek Infant Car Seat Cover Black
Levana Safe N'See Digital Video Baby Monitor with Talk-to-Baby Intercom and Lullaby Control (LV-TW501)
Snuza Portable Baby Movement Monitor
Fisher-Price Ocean Wonders Aquarium Bouncer
VTech Communications Safe &amp; Sounds Full Color Video and Audio Monitor
Safety 1st High-Def Digital Monitor
Chicco Cortina KeyFit 30 Travel System in Adventure
Prince Lionheart Warmies Wipes Warmer
Valco Baby Tri-mode Twin Stroller EX- Hot Chocolate
Adiri BPA Free Natural Nurser Ultimate Bottle Stage 1 White, Slow Flow (0-3 months)
Munchkin Nursery Projector and Sound System, White
The First Years True Choice P400 Premium Digital Monitor, 2 Parent Unit
Nuby Natural Touch Silicone Travel Infa Feeder, Colors May Vary, 3 Ounce
Peg-Perego Tatamia High Chair, White Latte
Fisher-Price Royal Potty
Safety 1st Exchangeable Tip 3 in 1 Thermometer
Safety 1st Lift Lock and Swing Gate
Evenflo Take Me Too Premiere Tandem Stroller - Castlebay
Cloth Dia

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by


$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [47]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    ## YOUR CODE HERE
    predictions = model.predict(data)
    
    # Compute the number of correctly classified examples
    ## YOUR CODE HERE
    countCorrect = 0
    for i in range(len(predictions)):
        if predictions[i] == true_labels[i]:
            countCorrect += 1

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy = countCorrect*1.0/len(predictions)
    return accuracy
print "function get_classification_accuracy loaded."

function get_classification_accuracy loaded.


Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [48]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.9145368370530358

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

accuracy of the sentiment_model on the test_data = 0.91   

Does a higher accuracy value on the training_data always imply that the classifier is better? No. Overfitting syndrome.

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subset of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [52]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [53]:
len(significant_words)

20

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [54]:
train_data['word_count_subset'] = train_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)
test_data['word_count_subset'] = test_data['word_count'].dict_trim_by_keys(significant_words, exclude=False)

Let's see what the first example of the dataset looks like:

In [55]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [56]:
print train_data[0]['word_count']

{'and': 3L, 'love': 1L, 'it': 3L, 'highly': 1L, 'osocozy': 1L, 'bags': 1L, 'leak': 1L, 'moist': 1L, 'does': 1L, 'recommend': 1L, 'was': 1L, 'wipes': 1L, 'disappointed': 1L, 'early': 1L, 'not': 2L, 'now': 1L, 'holder': 1L, 'wipe': 1L, 'keps': 1L, 'wise': 1L, 'i': 1L, 'planet': 1L, 'my': 2L, 'came': 1L}


Since we are only working with a subset of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [57]:
print train_data[0]['word_count_subset']

{'love': 1L, 'disappointed': 1L}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [58]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'sentiment',
                                                   features=['word_count_subset'],
                                                   validation_set=None)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.803722     | 0.866815          |

| 6         | 7        | 0.941847     | 0.866815          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 21
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 20

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.9799

Settings
--------
Log-likelihood                 : 44323.7254

Highest Positive Coefficients
-----------------------------
word_count_subset[loves]       : 1.6773
word_count_subset[perfect]     : 1.5145
word_count_subset[love]        : 1.3654
(intercept)                    : 1.2995
word_count_subset[easy]        : 1.1937

Lowest Negative Coefficients
----------------------------
word_count_subset[disappointed] : -2.3551
wo

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

note: len(significant_words) = 20
Number of coefficients         : 21
Number of unpacked features    : 20
ie: coefficient[0] will be the intercept or constant


In [59]:
print type(simple_model.coefficients)
print simple_model.coefficients.column_names()
print simple_model.coefficients.shape

<class 'graphlab.data_structures.sframe.SFrame'>
['name', 'index', 'class', 'value', 'stderr']
(21, 5)


In [60]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.8693004559635229

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [61]:
simple_model.coefficients

name,index,class,value,stderr
(intercept),None,1,1.2995449552,0.0120888541331
word_count_subset,disappointed,1,-2.35509250061,0.0504149888557
word_count_subset,love,1,1.36543549368,0.0303546295109
word_count_subset,well,1,0.504256746398,0.021381300631
word_count_subset,product,1,-0.320555492996,0.0154311321362
word_count_subset,loves,1,1.67727145556,0.0482328275384
word_count_subset,little,1,0.520628636025,0.0214691475665
word_count_subset,work,1,-0.621700012425,0.0230330597946
word_count_subset,easy,1,1.19366189833,0.029288869202
word_count_subset,great,1,0.94469126948,0.0209509926591


Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [62]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

In [63]:
#alternate method to select the coefficients with the most positive effect on the sentiment.
#https://turi.com/products/create/docs/generated/graphlab.SFrame.topk.html
simple_model.coefficients.topk('value', k=21, reverse=False).print_rows(num_rows=21)
#redundant since same answer, selecting all rows (not subset of rows) and speed not an issue for this use case.

+-------------------+--------------+-------+-----------------+-----------------+
|        name       |    index     | class |      value      |      stderr     |
+-------------------+--------------+-------+-----------------+-----------------+
| word_count_subset |    loves     |   1   |  1.67727145556  | 0.0482328275384 |
| word_count_subset |   perfect    |   1   |  1.51448626703  |  0.049861952294 |
| word_count_subset |     love     |   1   |  1.36543549368  | 0.0303546295109 |
|    (intercept)    |     None     |   1   |   1.2995449552  | 0.0120888541331 |
| word_count_subset |     easy     |   1   |  1.19366189833  |  0.029288869202 |
| word_count_subset |    great     |   1   |  0.94469126948  | 0.0209509926591 |
| word_count_subset |    little    |   1   |  0.520628636025 | 0.0214691475665 |
| word_count_subset |     well     |   1   |  0.504256746398 |  0.021381300631 |
| word_count_subset |     able     |   1   |  0.191438302295 | 0.0337581955697 |
| word_count_subset |     ol

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [64]:
#print "simple_model.coefficients['value'] = \n", simple_model.coefficients['value']
predictions = simple_model.coefficients['value'][1:]
#NB : exclude 0th element (represents intercept term)
#print "type(predictions) = ", type(predictions) #<class 'graphlab.data_structures.sarray.SArray'>
result = simple_model.coefficients
positiveValueCount = 0
for i in range(predictions.shape[0]):
    if predictions[i]>0:
        positiveValueCount += 1
print "simple_model.coefficients.shape[0] = ", simple_model.coefficients.shape[0]
print "positiveValueCount = ", positiveValueCount

simple_model.coefficients.shape[0] =  21
positiveValueCount =  10


In [71]:
positive_significant_words=[]
for i in range(1,len(simple_model.coefficients)):
    #NB: need to exclude 0th element (intercept/constant)
    if simple_model.coefficients[i]['value'] > 0:
        positive_significant_words.append(simple_model.coefficients[i]['index'])
print "positive_significant_words= ", positive_significant_words
print "positive coefficients for the simple_model = ", len(positive_significant_words)
print type(positive_significant_words)

positive_significant_words=  ['love', 'well', 'loves', 'little', 'easy', 'great', 'able', 'perfect', 'old', 'car']
positive coefficients for the simple_model =  10
<type 'list'>


**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

In [98]:
sentModCoeff = sentiment_model.coefficients
print type(sentModCoeff)
print type(sentModCoeff['index'])
print sentModCoeff['index'].shape
count = 0
for word in positive_significant_words:
    #print "word=", word
    if sentModCoeff[sentModCoeff['index']==word]:
        #print sentModCoeff[sentModCoeff['index']==word]
        #print "is in sentModCoeff['index']"
        count += 1
    else:
        print word, " is not in sentModCoeff['index']"
if count == len(positive_significant_words):
    print "all words in positive_significant_words are in sentiment_model.coefficients.['index']"
else:
    print "NOT all words from positive_significant_words are in sentiment_model.coefficients.['index']"
#NB: this method is fast. uses SFrames to test for string in a column, much faster than alternatives.

<class 'graphlab.data_structures.sframe.SFrame'>
<class 'graphlab.data_structures.sarray.SArray'>
(121713,)
all words in positive_significant_words are in sentiment_model.coefficients.['index']


In [99]:
#alternate method for demonstration purposes. DO NOT USE - SLOW, locks up resources.
print sentiment_model.coefficients
print type(sentiment_model.coefficients)
print sentiment_model.coefficients.shape
sentiment_model_psw = []
numrows = sentiment_model.coefficients.shape[0]
print "numrows = ", numrows
#DO NOT RUN THIS LOOP_SLOW!!!!!
#for i in range(1,numrows):
#    print "@ i=", i, ", value = ", sentiment_model.coefficients[i]['value']
#    if sentiment_model.coefficients[i]['value']>0:
        #print sentiment_model.coefficients[i]['index']
        #print type(sentiment_model.coefficients[i]['index'])
#        sentiment_model_psw.append(sentiment_model.coefficients[i]['index'])
print "len(sentiment_model_psw) = ", len(sentiment_model_psw)
sentiment_model_psw = set(sentiment_model_psw)#convert list to set
positive_significant_words = set(positive_significant_words)#convert list to set
#now test if the set positive_significant_words is a true subset of sentiment_model_psw
positive_significant_words.issubset(sentiment_model_psw)
#NB: this method is ___SLOW___

+-------------+--------------+-------+------------------+--------+
|     name    |    index     | class |      value       | stderr |
+-------------+--------------+-------+------------------+--------+
| (intercept) |     None     |   1   |  1.30337080544   |  None  |
|  word_count |  recommend   |   1   |  0.303815600015  |  None  |
|  word_count |    highly    |   1   |  1.49183015276   |  None  |
|  word_count | disappointed |   1   |  -3.95748618393  |  None  |
|  word_count |     love     |   1   |  1.43301685439   |  None  |
|  word_count |      it      |   1   | 0.00986646490307 |  None  |
|  word_count |    planet    |   1   | -0.797764553926  |  None  |
|  word_count |     and      |   1   |  0.048449573172  |  None  |
|  word_count |     bags     |   1   |  0.165541436615  |  None  |
|  word_count |    wipes     |   1   | -0.0949937947269 |  None  |
+-------------+--------------+-------+------------------+--------+
[121713 rows x 5 columns]
Note: Only the head of the SFrame is

False

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [100]:
print "test"

test


In [101]:
#recall def get_classification_accuracy(model, data, true_labels)
#NB: true_labels = train_data['sentiment']
print "calculating...."
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

calculating....


0.979440247046831

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [103]:
print "calculating...."
get_classification_accuracy(simple_model, train_data, train_data['sentiment'])

calculating....


0.8668150746537147

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

based on train_data  
sentiment_model : 0.979440247046831  
simple_model    : 0.8668150746537147  

sentiment_model has higher accuracy on the TRAINING set

Now, we will repeat this exercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [121]:
print "calculating...."
accuracy_sentiment_model_Test_Data = get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])
print "accuracy_sentiment_model_Test_Data =", accuracy_sentiment_model_Test_Data

 calculating....
accuracy_sentiment_model_Test_Data = 0.914536837053


Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [107]:
print "calculating...."
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

calculating....


0.8693004559635229

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

based on test_data  
sentiment_model :   0.9145368370530358
simple_model    :   0.8693004559635229

sentiment_model has higher accuracy on the TEST set

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [117]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print "num_positive = ", num_positive
print "num_negative = ", num_negative
print "num_positive_test + num_negative_test = ", num_positive_test+num_negative_test
print "train_data.shape[0] = ", train_data.shape[0]
print "majority class classifier for training data = ", num_positive / train_data.shape[0]

num_positive =  112164
num_negative =  21252
num_positive_test + num_negative_test =  33336
train_data.shape[0] =  133416
majority class classifier for training data =  0.840708760568


 majority class for train_data is num_positive  

Now compute the accuracy of the majority class classifier on test_data.

In [112]:
num_positive_test  = (test_data['sentiment'] == +1).sum()
print "num_positive_test = ", num_positive_test
num_negative_test  = (test_data['sentiment'] == -1).sum()
print "num_negative_test = ", num_negative_test
print "test_data.shape[0] = ", test_data.shape[0]
print "num_positive_test + num_negative_test = ", num_positive_test+num_negative_test

num_positive_test =  28095
num_negative_test =  5241
test_data.shape[0] =  33336
num_positive_test + num_negative_test =  33336


majority class for test_data is num_positive

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [122]:
#majority class classifier = num_positive_test / (test_data.shape[0])
print "majority class classifier on test data = ", num_positive_test / (test_data.shape[0])
print "accuracy_sentiment_model_Test_Data =", accuracy_sentiment_model_Test_Data
#recall
#sentiment_model = graphlab.logistic_classifier.create(train_data, 
#         target = 'sentiment', features=['word_count'], validation_set=None)

majority class classifier on test data =  0.842782577394
accuracy_sentiment_model_Test_Data = 0.914536837053


**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?